# EVALUACIÓN Y BENCHMARKING

### Para cada método de resolución de la W de los items anteriores, generar una matriz de confusión evaluando a partir de los pares de embeddings de validación o testing (Xv, Yv)

In [ ]:
import numpy as np
import alc
import pandas as pd
import main

Construimos las funciones que nos permitirán predecir la clase y ver la clase real, tal que:

- Si `pred[0] > pred[1]` → **gato**
- Si `pred[1] > pred[0]` → **perro**

In [ ]:
def predecir_clases(W, Xv):
    Y_pred = alc.multiplicacionMatricial(W, Xv)
    clases = np.zeros(Y_pred.shape[1], dtype=int)

    for i in range(Y_pred.shape[1]):
        if Y_pred[0, i] > Y_pred[1, i]:
            clases[i] = 0   # gato
        else:
            clases[i] = 1   # perro
    return clases

In [ ]:
def clases_reales(Y):
    clases = np.zeros(Y.shape[1], dtype=int)

    for i in range(Y.shape[1]):
        if Y[0, i] == 1:
            clases[i] = 0   # gato
        else:
            clases[i] = 1   # perro
    return clases

Ahora definimos la función que nos da la matriz de confusión, encargada de comparar las predicciones con los valores reales:
\begin{bmatrix}
TP & FP \\
FN & TN
\end{bmatrix}

In [ ]:
def matriz_confusion(clases_reales, clases_pred):
    M = np.zeros((2,2), dtype=int)
    for i in range(clases_reales.shape[0]):
        M[clases_reales[i], clases_pred[i]] += 1
    return M

Luego usamos esta matriz confusión para evaluar el desempeño de cada uno de nuestros métodos, tal como se ve a continuación

In [ ]:
def evaluar_metodo(W, Xv, Yv):
    true = clases_reales(Yv)
    pred = predecir_clases(W, Xv)
    return matriz_confusion(true, pred)

Entonces, utilizamos "evaluar_metodo" para cada uno de ellos; Ecuaciones Normales, Descomposición en Valores Singulares, Descomposición QR Householder y Descomposición QR Gram-Schmidt y juntamos nuestros resultados en un diccionario. Utilizaremos para testear pares (Xv, Yv) de nuestro archivo "obtener_embeddings.py"

In [ ]:
"CÓDIGO PARA OBTENER PARES DE TESTING"

In [ ]:
resultados = {}

W_en = pinvEcuacionesNormales(Xt, L, Yt)
M_en = evaluar_metodo(W_en, Xv, Yv)
resultados["Ecuaciones Normales"] = M_en

W_svd = pinvSVD(U, S, V, Yt)
M_svd = evaluar_metodo(W_svd, Xv, Yv)
resultados["SVD"] = M_svd

Q_hh, R_hh = xTraspuestaQR(Xt, "RH")
W_hh = pinvHouseHolder(Q_hh, R_hh, Yt)
M_hh = evaluar_metodo(W_hh, Xv, Yv)
resultados["QR Householder"] = M_hh

Q_gs, R_gs = xTraspuestaQR(Xt, "GS")
W_gs = pinvGramSchmidt(Q_gs, R_gs, Yt)
M_gs = evaluar_metodo(W_gs, Xv, Yv)
resultados["QR Gram-Schmidt"] = M_gs

Ahora definimos cómo calcular cada una de las métricas de rendimiento (Accuracy, Precisión, Recall y F1)

In [ ]:
def calcular_metricas(M):

    tp_g = M[0,0]
    fn_g = M[0,1]
    fp_g = M[1,0]
    tp_p = M[1,1]
    fn_p = fp_g       # real perro, pred gato
    fp_p = fn_g       # real gato, pred perro

    # ACCURACY
    accuracy = (tp_g + tp_p) / np.sum(M)

    # PRECISIÓN
    prec_g = tp_g / (tp_g + fp_g) if (tp_g + fp_g) > 0 else 0
    prec_p = tp_p / (tp_p + fp_p) if (tp_p + fp_p) > 0 else 0

    # RECALL
    rec_g = tp_g / (tp_g + fn_g) if (tp_g + fn_g) > 0 else 0
    rec_p = tp_p / (tp_p + fn_p) if (tp_p + fn_p) > 0 else 0

    # F1
    f1_g = 2 * prec_g * rec_g / (prec_g + rec_g) if (prec_g + rec_g) > 0 else 0
    f1_p = 2 * prec_p * rec_p / (prec_p + rec_p) if (prec_p + rec_p) > 0 else 0

    return {
        "Accuracy": accuracy,
        "Precisión Gato": prec_g,
        "Recall Gato": rec_g,
        "F1 Gato": f1_g,
        "Precisión Perro": prec_p,
        "Recall Perro": rec_p,
        "F1 Perro": f1_p,
    }

### Presentar una tabla comparativa de los resultados de cada metodología donde en las columnas se debe mostrar la performance en clasificación

In [ ]:
tabla = pd.DataFrame({
    metodo: {
        "Accuracy": met["Accuracy"],
        "Precisión Gato": met["Precisión Gato"],
        "Recall Gato": met["Recall Gato"],
        "F1 Gato": met["F1 Gato"],
        "Precisión Perro": met["Precisión Perro"],
        "Recall Perro": met["Recall Perro"],
        "F1 Perro": met["F1 Perro"]
    }
    for metodo, met in resultados.items()
}).T

tabla

# SÍNTESIS FINAL